In [1]:
import pandas as pd
import sqlite3 

## INSTANCIA DE BBDD ##

In [2]:
# Conectamos con la base de datos .db. ESTA PARTE SIEMPRE HAY QUE HACERLA
connection = sqlite3.connect("data/Bar_TheBridge.db")

# Obtenemos un cursor que utilizaremos para hacer las queries. Es lo mismo al cntr enter en el dbeaver. PODEMOS UTILIZAR ESTE O PANDAS LO CREA, PERO MEJOR HACERLO.
crsr = connection.cursor()

### Función para interactuar con la BBDD ###

In [10]:
# Con esta función leemos los datos y lo pasamos a un DataFrame de Pandas
def sql_query(query):

    # Ejecuta la query
    crsr.execute(query)

    # Almacena los datos de la query 
    ans = crsr.fetchall()
# añadir un try except, por si no hay nada en la tabla
    # Obtenemos los nombres de las columnas de la tabla
    names = [description[0] for description in crsr.description]

    return pd.DataFrame(ans,columns=names)



Podríamos utilizar pandas en lugar de esta función, pero sería más complicado:
- pd.read_sql(query, conexión, ...)
- pd.read_sql_query()

## Creación de tablas ##

### 1. Comprobamos si la tabla ya existe, el codigo de abajo muestra las tablas que hay en la BBDD:###

In [79]:
res = crsr.execute("SELECT name FROM sqlite_master WHERE type='table'")
for name in res:
    print(name[0])

cerveza
bares
EMPLEADOS
REPARTO


### 2. Creamos la tabla CERVEZA ###

Escribimos los comandos de SQL en mayus y el resto en minus.
0 ANTES DE UN NUMERO SIEMPRE ES STRING


In [80]:
query_cerveza = """ 
CREATE TABLE cerveza(
    CodC VARCHAR(3),
    Envase VARCHAR(50), 
    Capacidad FLOAT,
    Stock INT
    )"""

crsr.execute(query_cerveza)


OperationalError: table cerveza already exists

INT VA DESDE -2M a 2M

### 2. Creamos la tabla BARES ###

In [22]:
query_bares = """ 
CREATE TABLE bares(
    CodB VARCHAR(4),
    Nombre VARCHAR(50), 
    Cif VARCHAR(10),
    Localidad VARCHAR(50)
    )"""

# crsr.execute(query_bares)


### 2. Creamos la tabla EMPLEADOS ###

In [81]:
query_empleados = """ 
CREATE TABLE EMPLEADOS(
    CodE VARCHAR(2),
    Nombre VARCHAR(50), 
    Sueldo INT
        )"""
crsr.execute(query_empleados)


OperationalError: table EMPLEADOS already exists

### 2. Creamos la tabla REPARTO ###

In [82]:
query_reparto = """ 
CREATE TABLE REPARTO(
    CodE VARCHAR(2),
    CodB VARCHAR(4), 
    CodC VARCHAR(3),
    Fecha DATE,
    Cantidad INT
        )"""

crsr.execute(query_reparto)

OperationalError: table REPARTO already exists

Para relacionar las bbdd hay que añadir una linea de codigo que indique  las pk y las fk de cada tabla

## Rellenar tablas ##

1. A través de una query, si falta algún dato ,deberia rellenarlo con nan, pero no en este tipo de insert

In [83]:
# Insertamos datos en la tabla cerveza 
query_insert = """ 
INSERT INTO cerveza  VALUES
('01', 'Botella', 0.2, 3600),
('02', 'Botella', 0.33, 1200),
('03', 'Lata', 0.33, 2400),
('04', 'Botella', 1, 288),
('05', 'Barril', 60, 30)
"""
crsr.execute(query_insert)

JAMAS UTILIZAR DELETE FROM TABLE SIN UN WHERE porque se borra la base de datos ENTERA.

En python sí podemos meter datos duplicados porque SE CREA UN INDICE , que es diferente para cada fila.


In [84]:
# Insertamos datos en la tabla Reparto: ahora indicamos el orden de las columnas en las que voy a insertar datos

query_insert2 = """
INSERT INTO reparto (CodE, CodB, CodC, Fecha, Cantidad) VALUES
('1','001','01','2005-10-21',240),
('1','001','02','2005-10-21',48),
('1','002','03','2005-10-22',60),
('1','004','05','2005-10-22',4),
('2','002','03','2005-10-22',48),
('2','002','05','2005-10-23',2),
('2','004','01','2005-10-23',480),
('2','004','02','2005-10-24',72),
('3','003','03','2005-10-24',48),
('3','003','04','2005-10-25',20)

"""

crsr.execute(query_insert2)

In [85]:
df = pd.read_sql('SELECT * FROM reparto', connection, parse_dates=['Fecha'])
# no está leyendo bien los datos de date.
# AL METER LOS DATOS DE LA QUERY EN UN DF, pandas lee de la manera que le sea más sencilla, asi que aunque cambiamos las fechas, no lo reconoce aun.
# Hay que añadir en el read_sql el parse dates.

In [56]:
# crsr.execute('DELETE FROM reparto WHERE CodB ="001"'')

In [48]:
# query_delete = 'DELETE FROM reparto WHERE CodB ="001"'
crsr.execute(query_delete)
# esta y la anterior, son iguales. 

2. Através de un DataFrame

In [86]:
# Insertamos datos en la tabla Bares: ahora no lo hacemos con query, si no creando un DF

bares= pd.DataFrame(
    {"CodB": ['001','002','003','004'],
     "nombre": ['Stop','Las Vegas','Club Social','Otra Ronda'],
     "Cif": ['11111111X','22222222Y','None','33333333Z'],
     "Localidad": ['Villa Botijo','Villa Botijo','Las Ranas','La Esponja']
    })

bares

,CodB,nombre,Cif,Localidad
0,001,Stop,11111111X,Villa Botijo
1,002,Las Vegas,22222222Y,Villa Botijo
2,003,Club Social,None,Las Ranas
3,004,Otra Ronda,33333333Z,La Esponja


In [87]:
bares.to_sql(name='bares', con=connection, if_exists='append',index=False)
# nos da un error la primera vez, porque la tabla que hemos creado no tiene columna index 

ValueError: Table 'bares' already exists.

In [88]:
# Insertamos datos en la tabla Bares: ahora no lo hacemos con query, si no creando un DF

empleados= pd.DataFrame(
    {"CodE": ['1','2','3'],
     "Nombre": ['Prudencio Caminero','Vicente Merario','Valentin Siempre'],
     "Sueldo": [120000,110000,100000]
    })

empleados
empleados.to_sql(name='EMPLEADOS', con=connection,if_exists='append', index=False)


ValueError: Table 'EMPLEADOS' already exists.

In [68]:
# GUARDAR CAMBIOS: 1 query, 2 ejecutar cursor, 3 guardar la conexion. 
connection.commit()
connection.close()
# hacemos que todo se ejecute y cerramos la conexión para que otros la puedan utilizar

## RESPUESTAS ##


2. Obtener  el   Cif  y  nombre  de  los  bares  a  los  que  se  ha  repartido  cerveza  de  tipo  Botella  y  
capacidad inferior a 1 litro, ordenados por localidad 
3. Obtener  los  repartos  (nombre  del  bar,  envase  y  capacidad  de  la  bebida, fecha y cantidad) 
realizados por Prudencio Camionero. 
4. Obtener  los  bares  a  los  que  se  les  ha  repartido  envases  de  tipo  botella  y  capacidad  0.2  ó  
0.33 
5. Nombre de los empleados que han repartido a los bares "Stop" y "Las Vegas" cervezas con 
envase botella. 
6. Obtener  el  nombre  y  número  de  viajes  que  ha  realizado  cada  empleado  fuera  de  Villa  
Botijo. 
7. Obtener el nombre y localidad del bar que más litros de cerveza ha comprado.  
8. Obtener  los  bares  que  han  adquirido  todos  los  tipos  de  cerveza  con  envase  de  botella   y 
capacidad menor que 1 litro.  
9. Subir un 5% el sueldo del empleado que más días haya trabajado. 
10. Insertar  un  nuevo  reparto  del  empleado  “Vicente Merario” al bar  “Stop” de 48 cervezas de tipo lata el 
día 10/26/05. 

In [70]:
# Conectamos con la base de datos .db. ESTA PARTE SIEMPRE HAY QUE HACERLA
connection = sqlite3.connect("data/Bar_TheBridge.db")

# Obtenemos un cursor que utilizaremos para hacer las queries. Es lo mismo al cntr enter en el dbeaver. PODEMOS UTILIZAR ESTE O PANDAS LO CREA, PERO MEJOR HACERLO.
crsr = connection.cursor()

In [92]:
# 1. Obtener  el  nombre    de  los  empleados   que  hayan  repartido  al  bar  Stop  durante la semana 
# del 17 al 23 de octubre de 2005. 

query1 = '''
SELECT  DISTINCT e.Nombre
FROM EMPLEADOS as e
INNER JOIN reparto as r
ON e.CodE = r.CodE
INNER JOIN bares as b
ON b.CodB = r.CodB
WHERE b.Nombre = 'Stop' AND r.Fecha BETWEEN '2005-10-17' AND '2005-10-23'
'''


# HAY QUE USAR DISTINCT PARA QUE ME DE SOLO EL NOMBRE, YA QUE EN REPARTO EL MISMO SALE VARIAS VECES


empleados1 = sql_query(query1)
empleados1

,Nombre
0,Prudencio Caminero
